# Análisis de Base de Datos SQLite

Este notebook analiza la base de datos SQLite ubicada en:
`C:\Users\cpascual\PycharmProjects\pythonProject\cursos_actividades\sina\sqlite-analyzer\sqlite-analyzer\src\db\database.sqlite3.db`

Verificaremos:
1. La estructura de la base de datos (tablas y columnas)
2. Si las tablas contienen información o están vacías
3. Proporción de valores NULL en cada tabla/columna

In [1]:
# Importar librerías necesarias
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuración de visualización
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [12]:
# Definir la ruta a la base de datos
db_path = r"C:\Users\cpascual\PycharmProjects\pythonProject\cursos_actividades\sina\sqlite-analyzer\src\db\database_new.sqlite3.db"

# Verificar que el archivo existe
if os.path.exists(db_path):
    print(f"✅ Base de datos encontrada: {db_path}")
    print(f"Tamaño: {os.path.getsize(db_path) / 1024:.2f} KB")
else:
    print(f"❌ Error: La base de datos no existe en la ruta especificada")

✅ Base de datos encontrada: C:\Users\cpascual\PycharmProjects\pythonProject\cursos_actividades\sina\sqlite-analyzer\src\db\database_new.sqlite3.db
Tamaño: 27952.00 KB


In [13]:
# Conectar a la base de datos
try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    print("✅ Conexión exitosa a la base de datos")
except Exception as e:
    print(f"❌ Error al conectar a la base de datos: {e}")

✅ Conexión exitosa a la base de datos


In [14]:
# Obtener lista de todas las tablas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

if tables:
    print(f"Se encontraron {len(tables)} tablas en la base de datos:")
    for i, table in enumerate(tables, 1):
        print(f"  {i}. {table[0]}")
else:
    print("No se encontraron tablas en la base de datos")

Se encontraron 236 tablas en la base de datos:
  1. ACCI_PATIENT_CONDITIONS
  2. ACCI_VEHICLE_TYPES
  3. ACCI_WORKER_TYPES
  4. ACTI_ACTIVITY_STATES
  5. ALLE_ALLERGY_CATEGORIES
  6. ALLE_ALLERGY_END_REASONS
  7. ALLE_ALLERGY_RESULTS
  8. ALLE_ALLERGY_SEVERITY_LEVELS
  9. ALLE_ALLERGY_TYPES
  10. ALLE_INTOLERANCES
  11. ALLE_L2_NOT_MED_ALLERGENS
  12. ALLE_NOT_MEDICINAL_ALLERGENS
  13. ALLE_UNKNOWN_REASONS
  14. APPO_ABSENT_REASONS
  15. APPO_ADMISSION_PREFERENCES
  16. APPO_ADMISSION_TYPES
  17. APPO_AMBI_STATES
  18. APPO_APPOINTMENTS
  19. APPO_APPOINTMENT_ATTENDANCES
  20. APPO_CANCELLATION_REASONS
  21. APPO_FREQUENCY
  22. APPO_INTERCONSULTATION_REASONS
  23. APPO_INTERCONSULTATION_SYSTEMS
  24. APPO_INTER_REJECT_REASON
  25. APPO_MOBILITIES
  26. APPO_NOT_ATTENDED_REASONS
  27. APPO_ORIGIN_TYPES
  28. APPO_PENDING_REVIEW_REASONS
  29. APPO_PERIODICS
  30. APPO_POSTPONED_REASONS
  31. APPO_PROPOSALS
  32. APPO_PROPOSAL_STATES
  33. APPO_PROPOSAL_TYPE_STATES
  34. APPO_PROP_ATTENT

## Análisis de Estructura y Contenido de Tablas

A continuación, analizaremos cada tabla para verificar:
1. Su esquema (columnas y tipos de datos)
2. Conteo total de registros
3. Porcentaje de valores NULL por columna
4. Muestra de datos (primeras 5 filas)

In [15]:
# Función para analizar una tabla
def analyze_table(table_name):
    print(f"\n{'='*80}")
    print(f"ANÁLISIS DE TABLA: {table_name}")
    print(f"{'='*80}")
    
    # Obtener esquema de la tabla
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns_info = cursor.fetchall()
    
    print("Estructura de la tabla:")
    print(f"{'Columna':<30} {'Tipo':<15} {'Nullable':<8} {'PK':<5}")
    print(f"{'-'*30} {'-'*15} {'-'*8} {'-'*5}")
    for col in columns_info:
        # col = (id, name, type, notnull, default_value, pk)
        print(f"{col[1]:<30} {col[2]:<15} {'No' if col[3] else 'Sí':<8} {'Sí' if col[5] else 'No':<5}")
    
    # Contar registros
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    
    if count == 0:
        print(f"\n⚠️ La tabla {table_name} está vacía (0 registros)")
        return
    
    print(f"\nNúmero total de registros: {count}")
    
    # Calcular porcentaje de NULL por columna
    null_stats = {}
    for col in columns_info:
        col_name = col[1]
        cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE {col_name} IS NULL")
        null_count = cursor.fetchone()[0]
        null_percentage = (null_count / count) * 100
        null_stats[col_name] = {
            'total': count,
            'null_count': null_count,
            'null_percentage': null_percentage
        }
    
    # Mostrar estadísticas de NULL
    print("\nEstadísticas de valores NULL:")
    print(f"{'Columna':<30} {'Total NULLs':<15} {'% NULLs':<10}")
    print(f"{'-'*30} {'-'*15} {'-'*10}")
    for col, stats in null_stats.items():
        print(f"{col:<30} {stats['null_count']:<15} {stats['null_percentage']:.2f}%")
    
    # Obtener una muestra de datos y mostrar información ampliada
    try:
        # Cargar todos los datos para análisis estadístico pero limitar a 1000 filas para evitar problemas de memoria
        df_full = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 1000", conn)
        df = df_full.head(5)  # Primeras 5 filas para la muestra
        
        print("\nMuestra de datos (primeras 5 filas):")
        print(df)
        
        # Verificar si todos los datos son NULL
        if df_full.isnull().all().all():
            print("⚠️ ADVERTENCIA: Todos los valores en la muestra son NULL")
            return
            
        # Mostrar estadísticas descriptivas básicas
        print("\nEstadísticas descriptivas:")
        
        # Para cada columna, mostrar estadísticas específicas según su tipo
        for col_name in df_full.columns:
            print(f"\n- Columna: {col_name}")
            
            # Detectar el tipo de columna
            if df_full[col_name].dtype.kind in 'iuf':  # Integer, Unsigned Integer, Float
                print(f"  Tipo: Numérico ({df_full[col_name].dtype})")
                print(f"  Mínimo: {df_full[col_name].min()}")
                print(f"  Máximo: {df_full[col_name].max()}")
                print(f"  Media: {df_full[col_name].mean():.2f}")
                print(f"  Mediana: {df_full[col_name].median()}")
                
                # Mostrar distribución de valores si hay menos de 10 valores distintos
                value_counts = df_full[col_name].value_counts().head(5)
                if len(value_counts) <= 10:
                    print(f"  Distribución de valores:")
                    for val, count in value_counts.items():
                        print(f"    {val}: {count} registros ({count/len(df_full)*100:.2f}%)")
            
            # Para columnas de texto
            elif df_full[col_name].dtype == 'object':
                print(f"  Tipo: Texto/Objeto ({df_full[col_name].dtype})")
                
                # Contar valores únicos
                unique_count = df_full[col_name].nunique()
                print(f"  Valores únicos: {unique_count}")
                
                # Si hay pocos valores únicos, mostrarlos todos (hasta 10)
                if 1 < unique_count <= 10:
                    value_counts = df_full[col_name].value_counts()
                    print(f"  Distribución de valores:")
                    for val, count in value_counts.items():
                        print(f"    {val}: {count} registros ({count/len(df_full)*100:.2f}%)")
                
                # Para muchos valores únicos, mostrar solo los más frecuentes
                elif unique_count > 10:
                    top_values = df_full[col_name].value_counts().head(5)
                    print(f"  Valores más frecuentes:")
                    for val, count in top_values.items():
                        print(f"    {val}: {count} registros ({count/len(df_full)*100:.2f}%)")
                
                # Mostrar longitud media del texto si los valores son strings
                non_null = df_full[col_name].dropna()
                if non_null.shape[0] > 0 and all(isinstance(x, str) for x in non_null.head(5)):
                    avg_len = non_null.str.len().mean()
                    print(f"  Longitud media: {avg_len:.2f} caracteres")
            
            # Para fechas
            elif pd.api.types.is_datetime64_any_dtype(df_full[col_name]):
                print(f"  Tipo: Fecha/Hora ({df_full[col_name].dtype})")
                if df_full[col_name].count() > 0:
                    print(f"  Fecha más antigua: {df_full[col_name].min()}")
                    print(f"  Fecha más reciente: {df_full[col_name].max()}")
                    # Mostrar distribución por año si hay variedad de años
                    years = df_full[col_name].dt.year.value_counts().head(5)
                    if len(years) > 1:
                        print(f"  Distribución por año:")
                        for year, count in years.items():
                            print(f"    {year}: {count} registros ({count/len(df_full)*100:.2f}%)")
            
            # Para otros tipos
            else:
                print(f"  Tipo: Otro ({df_full[col_name].dtype})")
                print(f"  Valores únicos: {df_full[col_name].nunique()}")
        
        # Analizar relaciones con otras tablas si hay columnas con '_ID' que podrían ser claves foráneas
        fk_columns = [col for col in df_full.columns if '_ID' in col and col != f"{table_name.split('_')[0]}_ID"]
        if fk_columns:
            print("\nPosibles relaciones con otras tablas:")
            for fk_col in fk_columns:
                # Detectar tabla relacionada por el prefijo
                prefix = fk_col.split('_')[0]
                related_tables = [t[0] for t in tables if t[0].startswith(prefix)]
                
                if related_tables:
                    print(f"  Columna {fk_col} puede relacionarse con: {', '.join(related_tables)}")
                    
                    # Mostrar los valores más comunes de esta FK
                    top_fk_values = df_full[fk_col].value_counts().head(3)
                    if not top_fk_values.empty:
                        print(f"  Valores más comunes de {fk_col}:")
                        for val, count in top_fk_values.items():
                            print(f"    {val}: {count} registros ({count/len(df_full)*100:.2f}%)")
        
    except Exception as e:
        print(f"Error al obtener datos extendidos: {e}")

In [9]:
# Analizar cada tabla en la base de datos
for table in tables:
    analyze_table(table[0])


ANÁLISIS DE TABLA: ACCI_PATIENT_CONDITIONS
Estructura de la tabla:
Columna                        Tipo            Nullable PK   
------------------------------ --------------- -------- -----
APCO_ID                        tinyint         Sí       Sí   
APCO_DELETED                   bit             Sí       No   
APCO_DESCRIPTION_ES            nvarchar        Sí       No   
MTIME                          datetime2       Sí       No   

Número total de registros: 6

Estadísticas de valores NULL:
Columna                        Total NULLs     % NULLs   
------------------------------ --------------- ----------
APCO_ID                        0               0.00%
APCO_DELETED                   0               0.00%
APCO_DESCRIPTION_ES            0               0.00%
MTIME                          0               0.00%

Muestra de datos (primeras 5 filas):
   APCO_ID  APCO_DELETED APCO_DESCRIPTION_ES                       MTIME
0        1             0           Conductor  2000-11-30 23:

## Resumen del Análisis

Basándonos en los resultados anteriores, podemos resumir:

1. **Estado general de la base de datos**
   - Número total de tablas
   - Tablas completamente vacías
   - Tablas que contienen solo valores NULL

2. **Recomendaciones**
   - Si hay tablas vacías, podrían eliminarse o poblarse con datos relevantes
   - Si hay un alto porcentaje de valores NULL, considerar rediseñar el esquema o poblar esos campos

In [16]:
# Crear un resumen general
print("RESUMEN DE ANÁLISIS DE LA BASE DE DATOS")
print("="*40)

# Contar tablas vacías y con solo valores NULL
empty_tables = []
null_only_tables = []

for table in tables:
    table_name = table[0]
    
    # Verificar si está vacía
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    
    if count == 0:
        empty_tables.append(table_name)
        continue
    
    # Verificar si solo tiene valores NULL
    try:
        df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 100", conn)
        if df.isnull().all().all():
            null_only_tables.append(table_name)
    except:
        pass

print(f"Total de tablas: {len(tables)}")
print(f"Tablas vacías: {len(empty_tables)}")
if empty_tables:
    print(f"  - " + "\n  - ".join(empty_tables))
    
print(f"Tablas con solo valores NULL: {len(null_only_tables)}")
if null_only_tables:
    print(f"  - " + "\n  - ".join(null_only_tables))

# Cerrar conexión
conn.close()
print("\nConexión cerrada.")

RESUMEN DE ANÁLISIS DE LA BASE DE DATOS
Total de tablas: 236
Tablas vacías: 7
  - HEAL_DIABETES_INDICATORS
  - HEAL_HYPERTENS_INDICATORS
  - HIST_ANDROLOGICALS
  - HIST_PEDIATRICS
  - HIST_SOCIOECONOMICS
  - ONCO_CANCERS
  - ONCO_FAMILIAR_HISTORY
Tablas con solo valores NULL: 0

Conexión cerrada.
Total de tablas: 236
Tablas vacías: 7
  - HEAL_DIABETES_INDICATORS
  - HEAL_HYPERTENS_INDICATORS
  - HIST_ANDROLOGICALS
  - HIST_PEDIATRICS
  - HIST_SOCIOECONOMICS
  - ONCO_CANCERS
  - ONCO_FAMILIAR_HISTORY
Tablas con solo valores NULL: 0

Conexión cerrada.


## Conclusiones

Basado en nuestro análisis:

1. **Si encontramos tablas vacías o con solo valores NULL:**
   - Esto podría indicar que la base de datos no está correctamente inicializada
   - Puede ser necesario cargar datos de ejemplo o reales antes de continuar con el análisis
   - Es posible que la estructura esté definida pero faltan datos para poblarla

2. **Próximos pasos recomendados:**
   - Verificar el proceso de carga de datos
   - Revisar si hay scripts de inicialización que no se han ejecutado
   - Confirmar que la ruta de la base de datos es la correcta (podría estar apuntando a una versión vacía)

## Información Adicional para Mejorar el Análisis

El análisis anterior proporciona ahora información más detallada sobre:

1. **Estadísticas numéricas** para columnas con datos numéricos
   - Mínimo, máximo, media, mediana
   - Distribución de valores más comunes

2. **Análisis de texto** para columnas tipo string/objeto
   - Conteo de valores únicos
   - Valores más frecuentes
   - Longitud media de textos

3. **Análisis temporal** para columnas de fecha/hora
   - Rango de fechas (más antigua a más reciente)
   - Distribución por año cuando aplica

4. **Detección de relaciones** entre tablas
   - Identificación de posibles claves foráneas
   - Valores comunes en estas relaciones

Cada tabla ahora mostrará información más detallada para entender mejor la estructura y contenido de los datos, lo que facilita identificar posibles problemas o patrones interesantes.

# Análisis de Resultados

## Resumen de hallazgos principales

Después de analizar la base de datos `database_new.sqlite3.db`, podemos concluir:

1. **Estructura general**: 
   - La base de datos contiene 236 tablas en total
   - Es un esquema amplio que sugiere un sistema de información médica complejo

2. **Estado de los datos**:
   - La mayoría de las tablas (229 de 236, o 97%) contienen datos
   - 7 tablas están completamente vacías (sin registros)
   - No se encontraron tablas con registros que contengan solo valores NULL

3. **Tablas vacías**: Todas las tablas vacías parecen estar relacionadas con áreas médicas específicas:
   - Indicadores de salud: diabetes e hipertensión
   - Historiales médicos especializados: andrológicos, pediátricos y socioeconómicos
   - Información oncológica: registros de cánceres e historiales familiares

## ¿Faltan datos?

**Respuesta**: Sí, pero de forma limitada y en áreas específicas.

La base de datos está predominantemente poblada (97% de las tablas tienen datos). Las tablas que están vacías parecen pertenecer a módulos o funcionalidades específicas del sistema, posiblemente:

- Módulos que aún no han sido implementados
- Funcionalidades planificadas pero no utilizadas actualmente
- Áreas donde no se han migrado o importado los datos correspondientes

## Recomendaciones

1. **Para las tablas vacías**: 
   - Verificar si estas áreas son realmente necesarias para la funcionalidad actual del sistema
   - Si son necesarias, priorizar la carga de datos para estas tablas específicas
   - Considerar la eliminación de tablas que no se utilizarán en el futuro previsible

2. **Completitud de datos**:
   - La base de datos parece tener suficientes datos para ser funcional
   - Se recomienda realizar un análisis de calidad más profundo en las tablas principales para verificar la integridad referencial

3. **Próximos pasos**:
   - Identificar las tablas principales del sistema y verificar la calidad e integridad de sus datos
   - Examinar las relaciones entre tablas para asegurar que las referencias son válidas
   - Desarrollar un plan para poblar las tablas vacías si son relevantes para la funcionalidad del sistema

In [ ]:
# Analizar las relaciones entre tablas para verificar integridad referencial
# Específicamente para las tablas relacionadas con las áreas vacías

try:
    # Reconexión a la base de datos ya que cerramos la conexión anteriormente
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Lista de prefijos de las tablas vacías
    empty_prefixes = ['HEAL_', 'HIST_', 'ONCO_']
    
    print("ANÁLISIS DE RELACIONES PARA TABLAS VACÍAS")
    print("=" * 50)
    
    # Buscar tablas relacionadas que SÍ contengan datos
    related_tables = []
    
    for prefix in empty_prefixes:
        # Encontrar todas las tablas con este prefijo
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ?", (f"{prefix}%",))
        prefix_tables = [row[0] for row in cursor.fetchall()]
        
        print(f"\nAnálisis para prefijo: {prefix}")
        print(f"Tablas con este prefijo: {len(prefix_tables)}")
        
        # Verificar cuáles tienen datos y cuáles no
        tables_with_data = []
        tables_without_data = []
        
        for table in prefix_tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            count = cursor.fetchone()[0]
            
            if count > 0:
                tables_with_data.append((table, count))
            else:
                tables_without_data.append(table)
        
        print(f"- Tablas con datos: {len(tables_with_data)}")
        for table, count in tables_with_data:
            print(f"  * {table}: {count} registros")
            
        print(f"- Tablas vacías: {len(tables_without_data)}")
        for table in tables_without_data:
            print(f"  * {table}")
    
    # Cerrar la conexión
    conn.close()
    print("\nAnálisis completado.")
    
except Exception as e:
    print(f"Error durante el análisis de relaciones: {e}")

## Conclusión sobre la falta de datos

Basado en el análisis completo, podemos concluir:

1. **Estado general**: La base de datos está bien poblada en su mayor parte, con solo 7 tablas vacías de un total de 236.

2. **Naturaleza de las tablas vacías**: Las tablas vacías pertenecen a módulos específicos relacionados con:
   - Datos clínicos especializados (diabetes, hipertensión)
   - Historiales médicos específicos (andrología, pediatría)
   - Información oncológica

3. **Impacto en el sistema**:
   - Si estas tablas representan funcionalidades secundarias o complementarias, su ausencia probablemente no afecte la operación principal del sistema
   - Si son centrales para algunas funcionalidades específicas, esas características no serán utilizables

4. **Recomendación final**:
   - La base de datos parece ser funcional para la mayoría de los propósitos
   - Las áreas vacías probablemente representan módulos que aún no están en uso activo
   - Se recomienda priorizar la población de estas tablas solo si son necesarias para los casos de uso actuales o próximos del sistema